In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import TensorBoard
import os
import tempfile
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

NAME = "Covid-Detection-{}".format(int(time.time()))

### Importing and showing dataset from kaggle repository

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/UPC-IA-WV71-2021/IA_TF/main/Cleaned-Data.csv")
df.head()

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,...,Gender_Male,Gender_Transgender,Severity_Mild,Severity_Moderate,Severity_None,Severity_Severe,Contact_Dont-Know,Contact_No,Contact_Yes,Country
0,1,1,1,1,1,0,1,1,1,1,...,1,0,1,0,0,0,0,0,1,China
1,1,1,1,1,1,0,1,1,1,1,...,1,0,1,0,0,0,0,1,0,China
2,1,1,1,1,1,0,1,1,1,1,...,1,0,1,0,0,0,1,0,0,China
3,1,1,1,1,1,0,1,1,1,1,...,1,0,0,1,0,0,0,0,1,China
4,1,1,1,1,1,0,1,1,1,1,...,1,0,0,1,0,0,0,1,0,China


### Prepare dataset for input layer

In [3]:
cols_to_drop = ['None_Sympton','Age_0-9','Age_10-19','Age_20-24','Age_25-59','Age_60+','Gender_Female','Gender_Male','Gender_Transgender','Severity_Mild','Severity_Moderate','Severity_None','Severity_Severe','Contact_Dont-Know','Contact_No','Contact_Yes','Country']
df = df.drop(cols_to_drop, axis = 1)
df.head()

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,Infected
0,1,1,1,1,1,1,1,1,1,0
1,1,1,1,1,1,1,1,1,1,0
2,1,1,1,1,1,1,1,1,1,0
3,1,1,1,1,1,1,1,1,1,0
4,1,1,1,1,1,1,1,1,1,0


We confirm that the data in the dataset has purely numeric values.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316800 entries, 0 to 316799
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype
---  ------                   --------------   -----
 0   Fever                    316800 non-null  int64
 1   Tiredness                316800 non-null  int64
 2   Dry-Cough                316800 non-null  int64
 3   Difficulty-in-Breathing  316800 non-null  int64
 4   Sore-Throat              316800 non-null  int64
 5   Pains                    316800 non-null  int64
 6   Nasal-Congestion         316800 non-null  int64
 7   Runny-Nose               316800 non-null  int64
 8   Diarrhea                 316800 non-null  int64
 9   Infected                 316800 non-null  int64
dtypes: int64(10)
memory usage: 24.2 MB


To analize the dataset this is converted into an array

In [5]:
dataset = df.values
dataset

array([[1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 1, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int64)

In [6]:
X = dataset[: ,0:8]
Y = dataset[: ,9]

Now, we have six variables that will be used in the dataset:

X_train (70% of dataset, 5 input features)
X_val (15% of dataset, 5 input features)
X_test (15% of dataset, 5 input features)
Y_train (70% of dataset, 1 label features)
Y_val (15% of dataset, 1 label features)
Y_test (15% of dataset, 1 label features)